<a href="https://colab.research.google.com/github/Lomzem/LomzemStockQuant/blob/main/LomzemStocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import os
import pandas as pd
import datetime
import sys

wanted_sheet_path = '/content/drive/MyDrive/LomzemStockCharts/stocks_to_obtain.xlsx'
raw_stock_data_path = '/content/drive/MyDrive/LomzemStockCharts/rawstockdata'

def get_stock_data(date, ticker):
	api_url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey=IyEWxobQmnBJh0QwQeNOWH7QggrJGFjp'
	try:
		data = requests.get(api_url).json()
		stock_df = pd.DataFrame(data['results'])
		stock_df.rename(columns = {'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 'v': 'Volume', 't': 'Time'}, inplace=True)
		stock_df.insert(0, 'Ticker', ticker)
		stock_df.insert(1, 'FTime', stock_df.apply(lambda row: pd.to_datetime(row['Time']-1.44e+7, unit='ms'), axis=1))
		stock_df.to_excel(f'{raw_stock_data_path}/{date}-{ticker}.xlsx', index=False)
		print(f'Got {raw_stock_data_path}/{date}-{ticker}.xlsx')
	except KeyError:
		print('Wait a little longer')
		sys.exit()

def read_wanted_sheet():
	df = pd.read_excel(wanted_sheet_path)
	for index, row in df.iterrows():
		files = [file for file in os.listdir(raw_stock_data_path)]
		if not (f'{row["Date"].date()}-{row["Ticker"]}.xlsx') in files:
			if row['Date'].date() < datetime.date.today()-datetime.timedelta(days=1):
				get_stock_data(row['Date'].date(), row['Ticker'].upper())

read_wanted_sheet()

Got /content/drive/MyDrive/LomzemStockCharts/rawstockdata/2021-11-24-LGVN.xlsx
Wait a little longer


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
# import pandas as pd
import os

cum_stock_data = pd.DataFrame()

for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata'):
	df = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{file}')
	cum_stock_data = pd.concat([cum_stock_data, df])

print(cum_stock_data)
cum_stock_data.to_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx', index=False)

    Ticker               FTime  Volume       vw   Open  Close   High    Low  \
0     VERU 2022-04-11 06:30:00    7448   5.1801   4.91   5.47   5.48   4.55   
1     VERU 2022-04-11 06:31:00    6869   5.3946   5.20   5.45   5.93   5.16   
2     VERU 2022-04-11 06:32:00   49308   6.0368   5.42   5.72   6.53   5.42   
3     VERU 2022-04-11 06:33:00   38057   5.6815   5.73   5.71   5.90   5.50   
4     VERU 2022-04-11 06:34:00   29283   5.6577   5.75   5.70   5.83   5.51   
..     ...                 ...     ...      ...    ...    ...    ...    ...   
827   LGVN 2021-11-24 20:55:00   11561  29.8882  30.00  29.88  30.05  29.82   
828   LGVN 2021-11-24 20:56:00   12496  29.9550  29.91  30.04  30.06  29.87   
829   LGVN 2021-11-24 20:57:00   15450  29.9195  30.02  29.80  30.02  29.75   
830   LGVN 2021-11-24 20:58:00   21116  29.6869  29.90  29.50  29.90  29.50   
831   LGVN 2021-11-24 20:59:00   50844  28.6939  29.58  28.40  29.75  28.14   

              Time    n   a  op  
0    164967300000

In [3]:
import requests
import os
import pandas as pd
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx')
ddf = pd.DataFrame()

ddf = pd.concat([ddf, df[['Ticker', 'FTime']]])
ddf['FTime'] = ddf['FTime'].dt.date

# df.loc[192]
# Get Open
t_df = df.loc[df['FTime'].dt.time==datetime.time(9,30), ['Ticker', 'FTime', 'Open']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Close
t_df = df.loc[df['FTime'].dt.time==datetime.time(15,59), ['Ticker', 'FTime', 'Close']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['High'].transform(max) == t_df['High']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Low
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'Low']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['Low'].transform(min) == t_df['Low']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<=datetime.time(20,0))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df = t_df.groupby(['Ticker', 'FTime'])['Volume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Premarket Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'Volume': 'PVolume'}, inplace=True)
t_df = t_df.groupby(['Ticker', 'FTime'])['PVolume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Gap %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['Gap %'] = (ddf['Open'] / ddf.groupby('Ticker').shift(1)['Close'] - 1) * 100

# Get Volume Ratio
ddf['Vol Ratio'] = ddf['Volume']/ddf['PVolume']

# Get Premarket High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,00)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'High': 'PHigh'}, inplace=True)
idx = t_df.groupby(['Ticker', 'FTime'])['PHigh'].transform(max) == t_df['PHigh']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

ddf.rename(columns={'FTime': 'Date'}, inplace=True)

ddf
# print(ddf.head())
# print(ddf.groupby('Ticker')['Close'].shift(1).head())
# t_df

,Ticker,Date,Open,Close,High,Low,Volume,PVolume,Gap %,Vol Ratio,PHigh
0,ADN,2022-04-05,2.2200,2.0100,2.2800,1.9850,1249714,2330,NaN,536.357940,2.2600
1,ADN,2022-04-06,2.5800,3.6350,3.9800,2.4700,162103715,1049987,28.358209,154.386402,2.6600
2,ADN,2022-04-07,3.3200,3.3000,3.4500,3.0400,19160827,2447205,-8.665750,7.829678,3.8800
3,ADN,2022-04-08,3.3600,2.9401,3.5500,2.8000,9323953,404272,1.818182,23.063564,3.3900
4,ADN,2022-04-11,2.8100,2.6800,2.8300,2.5700,2888275,131866,-4.425020,21.903106,2.9200
5,AXSM,2022-04-19,40.2600,40.9200,41.6100,38.0500,11045166,2037628,NaN,5.420600,46.8000
6,CASA,2022-04-18,6.0400,7.1100,7.1800,5.6300,185110834,15728315,NaN,11.769273,6.0500
7,CASA,2022-04-19,6.5800,6.5400,6.9500,6.0200,37292653,4309616,-7.454290,8.653359,7.1600
8,CELZ,2022-03-22,1.7000,1.7000,1.8000,1.6300,469405,4660,NaN,100.730687,1.7500
9,CELZ,2022-03-23,3.4500,3.1100,3.9000,2.6300,140002440,21615566,102.941176,6.476927,3.5600
